In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import fasttext
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# NLP Preprocessing
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import preprocess_string

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [26]:
def split_data(dataframe, ylabel, test_size=0.2):
    
    training_df, test_df = train_test_split(
        dataframe,
        test_size=test_size,
        random_state=42,
        shuffle= True,
        stratify=dataframe[ylabel],
    )

    return training_df, test_df

financial_news = pd.read_csv("/kaggle/input/financial-phrasebank/phrasebank.csv")


le = LabelEncoder()
le.fit(financial_news["labels"])
financial_news["labels"] = le.transform(financial_news["labels"])

In [27]:
dataset_train, ds = split_data(financial_news, "labels")
dataset_train, dataset_val = split_data(dataset_train, "labels")

# NLP Preprocess
dataset_train.iloc[:, 0] = dataset_train.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
ds.iloc[:, 0] = ds.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))
dataset_val.iloc[:, 0] = dataset_val.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

# Prefixing each row of the category column with '__label__'
dataset_train.iloc[:, 1] = dataset_train.iloc[:, 1].apply(lambda x: '__label__' + str(x))
ds.iloc[:, 1] = ds.iloc[:, 1].apply(lambda x: '__label__' + str(x))
dataset_val.iloc[:, 1] = dataset_val.iloc[:, 1].apply(lambda x: '__label__' + str(x))

In [28]:
with open('fasttext_input_sentiment_train.txt', 'w') as f:
    for each_text, each_label in zip(dataset_train['titles'], dataset_train['labels']):
        f.writelines(f'{each_label} {each_text}\n')

with open('fasttext_input_sentiment_test.txt', 'w') as f:
    for each_text, each_label in zip(ds['titles'], ds['labels']):
        f.writelines(f'{each_label} {each_text}\n')
        
with open('fasttext_input_sentiment_val.txt', 'w') as f:
    for each_text, each_label in zip(dataset_val['titles'], dataset_val['labels']):
        f.writelines(f'{each_label} {each_text}\n')